In [1]:
import pandas as pd
flights_df=pd.read_csv('./plot_files/flights_df.zip',compression='zip', sep=',',keep_default_na=False, na_values=['', 'NaN'], index_col=0)
flights_df.head()

,airline_iata,acft_icao,acft_class,seymour_proxy,source,Seats,n_flights,iata_departure,iata_arrival,departure_lon,...,CO2 (Mt),ASK (Bn),Seats (Mn),CO2 Ppax,departure_continent_name,arrival_continent_name,departure_country_name,departure_ISO3,arrival_country_name,arrival_ISO3
0,5V,PC12,TP,PC12,BTS,4.5,0.5,05A,AET,-148.183889,...,1.960577e-07,1.007743e-06,0.000005,52.492030,North America,North America,United States of America,USA,United States of America,USA
1,5V,C208,TP,C208,BTS,4.5,0.5,05A,AKP,-148.183889,...,1.598180e-07,7.309065e-07,0.000005,42.789285,North America,North America,United States of America,USA,United States of America,USA
2,5V,PC12,TP,PC12,BTS,4.5,0.5,05A,AKP,-148.183889,...,1.655009e-07,7.309065e-07,0.000005,44.310814,North America,North America,United States of America,USA,United States of America,USA
3,5V,C208,TP,C208,BTS,4.5,0.5,05A,ARC,-148.183889,...,1.364440e-07,5.641599e-07,0.000005,36.531199,North America,North America,United States of America,USA,United States of America,USA
4,5V,PC12,TP,PC12,BTS,9.0,1.0,05A,CXF,-148.183889,...,2.634203e-07,8.423252e-07,0.000009,35.263764,North America,North America,United States of America,USA,United States of America,USA


In [2]:
flights_df=flights_df[flights_df.acft_class.isin(['NB','WB','TP','RJ','Unknown Aircraft'])]
flights_df.ASK.sum()

10663793075111.785

In [3]:
flights_EU_US=flights_df[flights_df.departure_continent.isin(['EU','NA']) | flights_df.arrival_continent.isin(['EU','NA'])]

Calibration on EU+US 

In [4]:
groups=flights_EU_US.groupby('acft_class')['ASK'].sum()/flights_EU_US.ASK.sum()*100
groups=groups.reset_index()

Asssuming unknwon acft are proportional to other categories

In [5]:
known_share=100-groups[groups.acft_class=='Unknown Aircraft']['ASK']
groups['ASK share']=groups[groups.acft_class!='Unknown Aircraft']['ASK']/(known_share.values[0])

In [6]:
groups

,acft_class,ASK,ASK share
0,NB,41.616663,0.470079
1,RJ,3.625191,0.040948
2,TP,0.344148,0.003887
3,Unknown Aircraft,11.468869,NaN
4,WB,42.945130,0.485085


In [7]:
#mapping based on paco vlue from planespotters
mapping = {
    'TP': 2257,
    'NB': 16257,
    'RJ': 3516,
    'WB': 5453
} 

groups['n_AC'] = groups['acft_class'].map(mapping)

In [8]:
groups['ASK_per_AC_per_year']=flights_df.ASK.sum()*groups['ASK share']/groups['n_AC']

In [9]:
groups

,acft_class,ASK,ASK share,n_AC,ASK_per_AC_per_year
0,NB,41.616663,0.470079,16257.0,3.083490e+08
1,RJ,3.625191,0.040948,3516.0,1.241932e+08
2,TP,0.344148,0.003887,2257.0,1.836661e+07
3,Unknown Aircraft,11.468869,NaN,NaN,NaN
4,WB,42.945130,0.485085,5453.0,9.486240e+08


In [10]:
SR=flights_EU_US[flights_EU_US.distance_km<=1500].groupby('acft_class')['ASK'].sum()/flights_EU_US[flights_EU_US.distance_km<=1500].ASK.sum()*100
MR=flights_EU_US[(flights_EU_US['distance_km'] > 1500) & (flights_EU_US['distance_km'] < 4000)].groupby('acft_class')['ASK'].sum()/flights_EU_US[(flights_EU_US['distance_km'] > 1500) & (flights_EU_US['distance_km'] < 4000)].ASK.sum()*100
LR=flights_EU_US[flights_EU_US.distance_km>4000].groupby('acft_class')['ASK'].sum()/flights_EU_US[flights_EU_US.distance_km>4000].ASK.sum()*100

In [11]:
ASK_AC_Y_SR = (
    groups[groups.acft_class=='NB']['ASK_per_AC_per_year'].values[0]*SR['NB']/(100-SR['Unknown Aircraft'])
    +groups[groups.acft_class=='WB']['ASK_per_AC_per_year'].values[0]*SR['WB']/(100-SR['Unknown Aircraft'])
    +groups[groups.acft_class=='RJ']['ASK_per_AC_per_year'].values[0]*SR['RJ']/(100-SR['Unknown Aircraft'])
    +groups[groups.acft_class=='TP']['ASK_per_AC_per_year'].values[0]*SR['TP']/(100-SR['Unknown Aircraft'])
              )

ASK_AC_Y_MR = (
    groups[groups.acft_class=='NB']['ASK_per_AC_per_year'].values[0]*MR['NB']/(100-MR['Unknown Aircraft'])
    +groups[groups.acft_class=='WB']['ASK_per_AC_per_year'].values[0]*MR['WB']/(100-MR['Unknown Aircraft'])
    +groups[groups.acft_class=='RJ']['ASK_per_AC_per_year'].values[0]*MR['RJ']/(100-MR['Unknown Aircraft'])
    +groups[groups.acft_class=='TP']['ASK_per_AC_per_year'].values[0]*MR['TP']/(100-MR['Unknown Aircraft'])
              )

ASK_AC_Y_LR = (
    groups[groups.acft_class=='NB']['ASK_per_AC_per_year'].values[0]*LR['NB']/(100-LR['Unknown Aircraft'])
    +groups[groups.acft_class=='WB']['ASK_per_AC_per_year'].values[0]*LR['WB']/(100-LR['Unknown Aircraft'])
    +groups[groups.acft_class=='RJ']['ASK_per_AC_per_year'].values[0]*LR['RJ']/(100-LR['Unknown Aircraft'])
    +groups[groups.acft_class=='TP']['ASK_per_AC_per_year'].values[0]*LR['TP']/(100-LR['Unknown Aircraft'])
              )

In [12]:
SR

acft_class
NB                  71.057245
RJ                  14.575748
TP                   1.790995
Unknown Aircraft    11.365064
WB                   1.210948
Name: ASK, dtype: float64

In [13]:
MR

acft_class
NB                  79.240104
RJ                   2.589020
TP                   0.004391
Unknown Aircraft    11.332583
WB                   6.833902
Name: ASK, dtype: float64

In [14]:
LR

acft_class
NB                   5.056164
RJ                   0.001666
TP                   0.000016
Unknown Aircraft    11.600186
WB                  83.341969
Name: ASK, dtype: float64

In [15]:
ASK_AC_Y_LR/1e10

0.09119867959230551

In [18]:
ASK_AC_Y_SR/1e6

280.95319142722946

In [19]:
ASK_AC_Y_MR/1e6

352.3056190515295

In [20]:
ASK_AC_Y_LR/1e6

911.9867959230552